In [4]:
import random
import time
import os
import pickle
import multiprocessing as mp
import uuid

def something_complex(var):
    time.sleep(0.0001)
    return var ** 2

import functools

def exec_time(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        t0 = time.time()
        value = func(*args, **kwargs)
        t1 = time.time()
        print(f'execution took {t1-t0:.2f}s')
        return value
    return wrapper_decorator

    
class Foo:
    _cache_property = None
    
    def __init__(self, *args, list_size, complexity=100000, **kwargs):
        self.list_size = list_size
        self.complexity = complexity
        self.cache_fname = f'cache-{complexity}.pkl'
        
    @property
    def input_list(self):
        # strategy for a property - very efficient but will never update. assumes that data never changes.
        if self._cache_property is None:
            self._cache_property = [int(random.random() * self.complexity) for i in range(self.list_size)]
            
        return self._cache_property
    
    def create_new_input_list(self):
        self._cache_property = None
        return self.input_list
    
    @exec_time
    def complex_loop(self):
        print('starting naive loop')
        result = 0
        for i in self.input_list:
            result += something_complex(i)
        
        return result
    
    @exec_time
    def make_complex_loop_faster(self):
        print('starting cached loop')      
        my_cache = {}
        result = 0
        for i in self.input_list:
            res = my_cache.get(i)
            if res is None:
                res =  something_complex(i)
                my_cache[i] = res
            result += res
        
        return result    
    
            

In [15]:
f = Foo(list_size=10000, complexity=100)

In [16]:
f.complex_loop()

starting naive loop
execution took 3.14s


32341861

In [17]:
f.make_complex_loop_faster()

starting cached loop
execution took 0.06s


32341861

## A more complex example: a persistent cache

In [2]:
import random
import time
import os
import pickle
import multiprocessing as mp
import uuid

def something_complex(var):
    time.sleep(0.0001)
    return var ** 2

import functools

def exec_time(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        t0 = time.time()
        value = func(*args, **kwargs)
        t1 = time.time()
        print(f'execution took {t1-t0:.2f}s')
        return value
    return wrapper_decorator

    
class Foo:
    _cache_property = None
    
    def __init__(self, *args, list_size, complexity=100000, **kwargs):
        self.list_size = list_size
        self.complexity = complexity
        self.cache_fname = f'cache-{complexity}.pkl'
        
    @property
    def input_list(self):
        # strategy for a property - very efficient but will never update. assumes that data never changes.
        if self._cache_property is None:
            self._cache_property = [int(random.random() * self.complexity) for i in range(self.list_size)]
            
        return self._cache_property
    
    def create_new_input_list(self):
        self._cache_property = None
        return self.input_list
    
    @exec_time
    def complex_loop(self):
        print('starting naive loop')
        result = 0
        for i in self.input_list:
            result += something_complex(i)
        
        return result
    
    @exec_time
    def make_complex_loop_faster(self):
        print('starting cached loop')      
        my_cache = {}
        result = 0
        for i in self.input_list:
            res = my_cache.get(i)
            if res is None:
                res =  something_complex(i)
                my_cache[i] = res
            result += res
        
        return result    
    
    def get_cache(self, delete_cache: bool=False):

        if delete_cache  and os.path.isfile(self.cache_fname):
            os.remove(self.cache_fname)
            
        if os.path.isfile(self.cache_fname):
            with open(self.cache_fname, 'rb') as pfile:
                my_cache = pickle.load(pfile)
        else:
            my_cache = {}
            
        return my_cache
        
    def dump_cache(self, my_cache):
        with open(self.cache_fname, 'wb') as pfile:
            pickle.dump(my_cache, pfile)
            
    def analyze_cache(self):
        cache = self.get_cache()
        uniques = len(set(cache.keys()))
        print(f'{uniques} keys in cache - max {self.complexity}')
        
    @exec_time
    def using_a_persistent_cache(self, delete_cache=False):
        print('starting (persistant) cached loop')        
            
        my_cache = self.get_cache(delete_cache=delete_cache)
            
        result = 0
        for i in self.input_list:
            res = my_cache.get(i)
            if res is None:
                res =  something_complex(i)
                my_cache[i] = res
            result += res
            
        
        self.dump_cache(my_cache)
          


            

In [125]:
def run(n):
    new = Bar(list_size=n)
    return new.using_a_distributed_persistent_cache()
    
    

In [134]:
pool = mp.Pool(processes=1)
processes = []
for i in range(10):
    processes.append(pool.apply_async(run, (10000000, )))

pool.close()
pool.join()
final = sum([p.get() for p in processes])

ForkPoolWorker-24] starting (distributed, persistant) cached loop
Cache loaded with 100000 items
empty cache, not writing
ForkPoolWorker-24] end
execution took 6.37s
ForkPoolWorker-24] starting (distributed, persistant) cached loop
Cache loaded with 100000 items
empty cache, not writing
ForkPoolWorker-24] end
execution took 6.47s
ForkPoolWorker-24] starting (distributed, persistant) cached loop
Cache loaded with 100000 items
empty cache, not writing
ForkPoolWorker-24] end
execution took 6.56s
ForkPoolWorker-24] starting (distributed, persistant) cached loop
Cache loaded with 100000 items
empty cache, not writing
ForkPoolWorker-24] end
execution took 6.41s
ForkPoolWorker-24] starting (distributed, persistant) cached loop
Cache loaded with 100000 items
empty cache, not writing
ForkPoolWorker-24] end
execution took 6.25s
ForkPoolWorker-24] starting (distributed, persistant) cached loop
Cache loaded with 100000 items
empty cache, not writing
ForkPoolWorker-24] end
execution took 6.62s
Fork

[5156101636,
 2040509584,
 489869689,
 2564409600,
 2630971849,
 866772481,
 10758400,
 293265625,
 220581904,
 1292043025]